Random Forest Regressor

In [33]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate, GridSearchCV,RandomizedSearchCV
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.inspection import permutation_importance
import shap


Random Forest Model Data Preparation



In [34]:
#load dat aset from csv source file
data_set = pd.read_csv('DataSets/RawDataWithStudentsRemoved.csv')

print(data_set.head())

#review data set structure
data_set.dtypes
print(data_set.shape)

#remove null term GPA from data set
data_set = data_set.dropna(subset = ['TermGPA'])
print(data_set.shape)

   FakeIdentifier  Age  Gender PrimaryMilitaryAffiliation AcademicCareer  \
0               1   18  Female    No Military Affiliation  Undergraduate   
1               2   72    Male    No Military Affiliation            Law   
2               3   26    Male    No Military Affiliation       Graduate   
3               4   47  Female                    Veteran       Graduate   
4               5   45    Male    No Military Affiliation  Undergraduate   

                          College  NumberofClassesEnrolled  TermGPA  \
0              College of Science                        4     3.75   
1   James E Rogers College of Law                        3     4.00   
2          College of Engineering                        2      NaN   
3       Coll of Ag Life & Env Sci                        2     4.00   
4  College of Information Science                        2     2.00   

   CumulativeGPA  UnitsPassedincludedinGPA  UnitsPassednotincludedinGPA  \
0          3.636                         

In [8]:
print(data_set['AcademicYear'].dtype)
print(data_set['TermGPA'].median())


int64
3.75


Let's also explore feature correlations to see if any of our non-target features correlate with each other. 

We may want to remove highly features which correlate with other non-target features

In [10]:
feature_corr_data_set = data_set.corr(numeric_only=True)

print(feature_corr_data_set)

                             FakeIdentifier       Age  \
FakeIdentifier                     1.000000  0.002807   
Age                                0.002807  1.000000   
NumberofClassesEnrolled           -0.000313 -0.167190   
TermGPA                           -0.002717  0.130831   
CumulativeGPA                     -0.004364  0.137993   
UnitsPassedincludedinGPA          -0.003381 -0.138960   
UnitsPassednotincludedinGPA        0.003713  0.022192   
AcademicYear                      -0.001265 -0.032702   

                             NumberofClassesEnrolled   TermGPA  CumulativeGPA  \
FakeIdentifier                             -0.000313 -0.002717      -0.004364   
Age                                        -0.167190  0.130831       0.137993   
NumberofClassesEnrolled                     1.000000  0.016221       0.005288   
TermGPA                                     0.016221  1.000000       0.853318   
CumulativeGPA                               0.005288  0.853318       1.000000   


We split the data set into a train and a test set (80/20) using the same random seed (698).
We decide which features to include and which to drop before training the model. We also must encode the remainig categorical features.

In [50]:
# Y target column
target_column = 'TermGPA'
test_percent = 0.20

#x_columns_to_drop = ['FakeIdentifier' , 'TermGPA']

#data set dropping the target feature
x_features = data_set.drop(['FakeIdentifier' ,
                            # 'College',
                             'PrimaryMilitaryAffiliation',
                             'CumulativeGPA',
                            # 'AcademicYear',
                            # 'Gender',
                            # 'AcademicCareer',
                            # 'UnitsPassedincludedinGPA',
                            # 'UnitsPassednotincludedinGPA',
                            # 'UAFullTimePartTime',
                            # 'NumberofClassesEnrolled',
                            # 'AcademicLevelEndofTerm',
                             'TermGPA'],
                               axis = 1)


#encoding categorical features
columns_to_encode = ['Gender',
                     #'PrimaryMilitaryAffiliation',
                     'AcademicCareer',
                     'College',
                     'UAFullTimePartTime',
                     'AcademicLevelEndofTerm',
                     'FirstGenerationFlag' 
                          ]


y_target = data_set[target_column]


#split the data into a 80/20 train test split using the same random seed for consistency
x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size = test_percent, random_state = 698) 

#encode categorical features, and drop one level to avoid multicolinearity
x_train_encoded = pd.get_dummies(x_train,
                                  columns=columns_to_encode,
                                    drop_first=True,
                                      dtype = int)
x_test_encoded = pd.get_dummies(x_test,
                                 columns=columns_to_encode,
                                 drop_first=True,
                                   dtype = int)
# restore missing features after encoding
x_train_features_encoded_cols = x_train_encoded.columns
x_test_encoded = x_test_encoded.reindex(columns=x_train_features_encoded_cols, fill_value=0)

#trying OneHOt encoder to help with missing features after encoding

#encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) 

# Fit the encoder on training data
#encoder.fit(x_train[columns_to_encode])

# Transform both training and test data
#onehot_encoded_train = encoder.transform(x_train[columns_to_encode])
#one_hot_encoded_test = encoder.transform(x_train[columns_to_encode])

# Convert to DataFrame for better readability (optional)
#encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(['color', 'size']))
#encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(['color', 'size']))



In [14]:
print(x_features)

print(x_train_encoded.columns)

print(x_test_encoded.columns)

       Age  CumulativeGPA FirstGenerationFlag
0       18          3.636                   N
1       72          3.833                   N
3       47          4.000                   N
4       45          3.250                   Y
5       22          4.000                   N
7       42          3.250                   Y
8       23          2.250                   N
9       38          3.136                   Y
10      21          2.484                   Y
11      46          3.356                   N
13      22          3.347                   N
14      39          3.786                   N
16      20          3.723                   N
18      27          3.667                   Y
19      23          1.250                   Y
21      31          4.000                   Y
23      32          4.000                   N
24      20          2.818                   Y
25      29          4.000                   N
26      25          3.000                   N
27      42          4.000         

In [45]:
#making the RF Regressor Model

metric = mean_squared_error
metric_name = "Mean Squared Error"
rfr_depth = 25
rfr_max_features = 0.35 # None, sqrt or log2
rfr_min_sample_split = 12
rfr_min_samples_leaf = 9
rfr_criterion = 'squared_error' #criterion{“squared_error”, “absolute_error”, “friedman_mse”, “poisson”}, default=”squared_error”
rfr_estimators = 200

model = RandomForestRegressor(n_estimators = rfr_estimators,
                               criterion = rfr_criterion,
                                 max_features = rfr_max_features,
                                 min_samples_leaf= rfr_min_samples_leaf,
                                    min_samples_split=rfr_min_sample_split,
                                      max_depth = rfr_depth,
                                          random_state=698)



In [38]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform, loguniform

In [40]:
#Parameter optimization

#for randomsearch to find a better set of parameters--not used in final run, hardcoded parm instead
rfr_optimized = RandomForestRegressor(random_state = 698)

param_grid = {
    'n_estimators':randint(low = 50, high = 300),
    'max_depth':randint(low=5, high=50),
    'min_samples_split':randint(low=2, high=16),
    'min_samples_leaf':randint(low=1, high=20),
    'max_features':uniform(loc=0.1, scale=0.9),
   
}


# Perform Random Search
random_search = RandomizedSearchCV(rfr_optimized, param_grid, cv=5, n_jobs = 2, scoring='neg_mean_squared_error', random_state= 698)
random_search.fit(x_train_encoded, y_train)

# Get the best parameters and model
best_CV_params = random_search.best_params_
best_CV_model = random_search.best_estimator_

print(f"{best_CV_params} are the best random cv parms" )
print(best_CV_model)



{'max_depth': 27, 'max_features': 0.2792263806438744, 'min_samples_leaf': 9, 'min_samples_split': 12, 'n_estimators': 230} are the best random cv parms
RandomForestRegressor(max_depth=27, max_features=0.2792263806438744,
                      min_samples_leaf=9, min_samples_split=12,
                      n_estimators=230, random_state=698)


{'max_depth': 33, 'max_features': 0.387746596652107, 'min_samples_leaf': 9, 'min_samples_split': 13, 'n_estimators': 198} are the best random cv parms
RandomForestRegressor(max_depth=33, max_features=0.387746596652107,
                      min_samples_leaf=9, min_samples_split=13,
                      n_estimators=198, random_state=698)

                      {'max_depth': 27, 'max_features': 0.2792263806438744, 'min_samples_leaf': 9, 'min_samples_split': 12, 'n_estimators': 230} are the best random cv parms
RandomForestRegressor(max_depth=27, max_features=0.2792263806438744,
                      min_samples_leaf=9, min_samples_split=12,
                      n_estimators=230, random_state=698)

# Evaluation

We review accuracy metrics ffrom a cross-fold validation, as well as the accuracy of the model run on our test set.

Wel also produce both e Feature Importances and the Permutation Importances for the model. 

In [18]:
#Adding kFold Cross validation--------------------------
kfold = KFold(n_splits=5, shuffle=True, random_state=698)


#for kfold accuracy scores
cv_scores = cross_val_score(model, x_train_encoded, y_train, cv=kfold, scoring='neg_mean_squared_error')

#review KFold Scores in the output (5-fold for our model)
print("Cross-validation scores:", cv_scores)

# Print the mean cross-validation score
print("Mean cross-validation score:", cv_scores.mean())

Cross-validation scores: [-0.39286605 -0.40539272 -0.40382745 -0.39670704 -0.41907263]
Mean cross-validation score: -0.40357317958538036


In [51]:
#fit the model
model.fit(x_train_encoded, y_train)

# Predict using test data
y_pred = model.predict(x_test_encoded) 

    # compare prediction to actual dev/test data
score = metric(y_test, y_pred)
print(f"{metric_name} on test set: {score:.4f}") #0.2409

r_squared = r2_score(y_test, y_pred)
print(f"r_squared on test set: {r_squared:.4f}") #0.8237

Mean Squared Error on test set: 0.5188
r_squared on test set: 0.6204


In [52]:
#feature importances:
feature_importances = model.feature_importances_ 
print(feature_importances)
    
    #put feature importances in a data frame
feature_names = x_train_encoded.columns 
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances}).sort_values(by = 'Importance', ascending = False)
pd.set_option('display.max_rows', None)
print(feature_importance_df)

[3.83249642e-02 2.47939713e-02 7.59511686e-01 1.57759737e-03
 1.67040759e-02 6.23357848e-03 2.06836872e-05 7.46356985e-04
 4.83639358e-02 1.86987360e-03 1.97881038e-05 1.70364367e-04
 6.63865634e-04 9.59205044e-05 1.38396753e-03 5.96645241e-03
 0.00000000e+00 6.15778776e-05 2.56379955e-03 3.00259500e-03
 2.67273611e-03 4.88440196e-04 6.27635724e-03 2.60399733e-04
 6.73893492e-05 6.95244790e-04 2.40973014e-03 5.83468100e-04
 8.02002571e-03 2.94942241e-02 5.39115840e-03 8.17054353e-04
 1.38765021e-02 6.60683557e-03 1.02653795e-02]
                                   Feature  Importance
2                 UnitsPassedincludedinGPA    0.759512
8             AcademicCareer_Undergraduate    0.048364
0                                      Age    0.038325
29         AcademicLevelEndofTerm_Graduate    0.029494
1                  NumberofClassesEnrolled    0.024794
4                             AcademicYear    0.016704
32           AcademicLevelEndofTerm_Senior    0.013877
34                   Firs

In [53]:
#permutation importance

result = permutation_importance(model, x_test_encoded, y_test, scoring = 'neg_mean_squared_error', n_repeats=10, random_state=698, n_jobs=-1)
sorted_idx = result.importances_mean.argsort()

permutation_importances = pd.Series(result.importances_mean[sorted_idx], index=x_test_encoded.columns[sorted_idx]).sort_values(ascending=False)
print("\nPermutation Importance:")
print(permutation_importances)


Permutation Importance:
UnitsPassedincludedinGPA                  1.417855
NumberofClassesEnrolled                   0.061105
AcademicCareer_Undergraduate              0.052366
AcademicLevelEndofTerm_Graduate           0.024109
AcademicLevelEndofTerm_Senior             0.021360
Age                                       0.017558
UAFullTimePartTime_P                      0.016963
College_Eller College of Management       0.010800
College_College of Information Science    0.008490
Gender_Male                               0.004681
AcademicLevelEndofTerm_Junior             0.003208
College_Undergraduate Education           0.002550
College_College of Science                0.002397
FirstGenerationFlag_Y                     0.002334
AcademicYear                              0.002211
AcademicLevelEndofTerm_Sophomore          0.001501
UnitsPassednotincludedinGPA               0.001165
College_College of Nursing                0.000982
College_College of Humanities             0.000561
Colleg

In [ ]:
#shap values

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_test_encoded)

In [ ]:
shap.summary_plot(shap_values, x_test_encoded, max_display = 25)
plt.title("SHAP Summary Plot")
plt.show()

In [ ]:
#feature correlations

x_feature_correlation = x_train_encoded.corr()
target_corr = pd.concat([x_train_encoded, y_train], axis = 1).corr()
#print(target_corr)
#-0.15 corr for first gen and term GPA



In [ ]:
#Look at decision trees...

In [ ]:
# scale the termGPA for training

#scaler = StandardScaler()
#y_train_reshape = y_train.values.reshape(-1, 1)

#y_train_zscore = scaler.fit_transform(y_train_reshape)

#y_test_zscore = 